# Import Comet for hyperparameter optimization

In [1]:
# import comet_ml at the top of your file
from comet_ml import Experiment
experiment = Experiment(
            project_name="music-genre-multiclass-classification",
            workspace="wodenwang820118",
        )
import comet_ml
import logging

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/f58994f9d39d4729bcce6e2658c91143



In [2]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

# Import libraries and data

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input,Embedding,Flatten,Concatenate,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model
%matplotlib inline
sns.set_style('whitegrid')

In [4]:
df_train_features = pd.read_csv('train_features.csv')
df_train_labels = pd.read_csv('train_labels.csv')

df_valid_features = pd.read_csv('valid_features.csv')
df_valid_labels = pd.read_csv('valid_labels.csv')

df_test_features = pd.read_csv('test_features.csv')

# Preprocess data

In [5]:
num_train_data = df_train_features.iloc[:,9:]
num_valid_data = df_valid_features.iloc[:,9:]
num_test_data = df_test_features.iloc[:,9:]

In [6]:
# num_train_data['loudness'] = df_train_features['loudness']
# num_train_data['tempo'] = df_train_features['tempo']
# num_train_data['duration'] = df_train_features['duration']

# num_valid_data['loudness'] = df_valid_features['loudness']
# num_valid_data['tempo'] = df_valid_features['tempo']
# num_valid_data['duration'] = df_valid_features['duration']

# num_test_data['loudness'] = df_test_features['loudness']
# num_test_data['tempo'] = df_test_features['tempo']
# num_test_data['duration'] = df_test_features['duration']

In [7]:
cat_train_data = df_train_features[['time_signature', 'key', 'mode']]
cat_valid_data = df_valid_features[['time_signature', 'key', 'mode']]
cat_test_data = df_test_features[['time_signature', 'key', 'mode']]

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [9]:
cat_train_data['time_signature_cat'] = le.fit_transform(cat_train_data['time_signature'])
cat_train_data['key_cat'] = le.fit_transform(cat_train_data['key'])
cat_train_data['mode_cat'] = le.fit_transform(cat_train_data['mode'])

cat_valid_data['time_signature_cat'] = le.fit_transform(cat_valid_data['time_signature'])
cat_valid_data['key_cat'] = le.fit_transform(cat_valid_data['key'])
cat_valid_data['mode_cat'] = le.fit_transform(cat_valid_data['mode'])

cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
cat_test_data['mode_cat'] = le.fit_transform(cat_test_data['mode'])

C:\Users\alex0\AppData\Local\Temp/ipykernel_14784/3557455032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_train_data['time_signature_cat'] = le.fit_transform(cat_train_data['time_signature'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_14784/3557455032.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_train_data['key_cat'] = le.fit_transform(cat_train_data['key'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_14784/3557455032.py:3: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [10]:
cat_train_data = cat_train_data.drop(['time_signature', 'key', 'mode'], axis=1)
cat_valid_data = cat_valid_data.drop(['time_signature', 'key', 'mode'], axis=1)
cat_test_data = cat_test_data.drop(['time_signature', 'key', 'mode'], axis=1)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [12]:
num_train = scaler.fit_transform(num_train_data)
num_valid = scaler.fit_transform(num_valid_data)
num_test = scaler.fit_transform(num_test_data)

In [13]:
num_train_y = pd.get_dummies(df_train_labels['genre'])
num_valid_y = pd.get_dummies(df_valid_labels['genre'])

# Categorical inputs

In [14]:
# size is the number of unique values in each characteristic
# size can be other values as well since it will generate numbers of vectors
time_signature_size = 6
key_size = 12
mode_size = 2

In [15]:
time_signature_input = Input(shape=(1,), name='time_signature')
key_input = Input(shape=(1,), name='key')
mode_input = Input(shape=(1,), name='mode')

In [16]:
# the min(600, round(1.6 * key_size**0.56)) is the rule of thumb for the input size(weight)
# from  fast.ai, Jeremy Howard
# reference from : https://bit.ly/3jsGw4u
time_signature_embedded = Embedding(cat_train_data.time_signature_cat.max()+1, min(600, round(1.6 * time_signature_size**0.56)), input_length=1, name='time_signature_embedded')(time_signature_input)
key_embedded = Embedding(cat_train_data.key_cat.max()+1, min(600, round(1.6 * key_size**0.56)),input_length=1,name='key_embedded')(key_input)
mode_embedded = Embedding(cat_train_data.mode_cat.max()+1, min(600, round(1.6 * mode_size**0.56)),input_length=1,name='mode_embedded')(mode_input)

In [17]:
cat_train = [cat_train_data.time_signature_cat,cat_train_data.key_cat,cat_train_data.mode_cat]
cat_valid = [cat_valid_data.time_signature_cat,cat_valid_data.key_cat,cat_valid_data.mode_cat]

# Weight adjustment and early stopping settings

In [18]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df_train_labels['genre']),y=df_train_labels['genre'])
class_weight = {i :class_weight[i] for i in range(8)}

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)

# Text model

In [20]:
import ktrain
from ktrain import text

In [21]:
def str_remove_comma(string):
    str = string.split(", ")
    return str

def list_to_string(list):
    str = " "
    return str.join(list)

In [22]:
# remove stopwords
import nltk
stopwords = nltk.corpus.stopwords.words('english')

In [23]:
x_train = df_train_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
y_train = df_train_labels['genre'].to_numpy()

x_test = df_valid_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
y_test = df_valid_labels['genre'].to_numpy()

In [24]:
x_train = x_train.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
x_test = x_test.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))

In [25]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [26]:
trn2, val2, preproc2 = text.texts_from_array(
    x_train=x_train, y_train=y_train,
    x_test=x_test, y_test=y_test,
    ngram_range=1, 
    maxlen=150,
    preprocess_mode='standard'
)

language: en
Word Counts: 4776
Nrows: 7678
7678 train sequences
train sequence lengths:
	mean : 52
	95percentile : 95
	99percentile : 146
x_train shape: (7678,150)
y_train shape: (7678, 8)
Is Multi-Label? False
450 test sequences
test sequence lengths:
	mean : 50
	95percentile : 101
	99percentile : 129
x_test shape: (450,150)
y_test shape: (450, 8)
task: text classification


In [27]:
model2 = text.text_classifier('fasttext', train_data=trn2, preproc=preproc2, metrics=['accuracy'])

Is Multi-Label? False
compiling word ID features...
maxlen is 150
done.


## The Dataset class for binding the audio data and the text data

In [28]:
import math
class MyCustomDataset(ktrain.SequenceDataset):
    def __init__(self, x, y, batch_size=32, shuffle=True):
        # error checks
        err = False
        if type(x) == np.ndarray and len(x.shape) != 2: err = True
        elif type(x) == list:
            for d in x:
                if type(d) != np.ndarray or len(d.shape) != 2:
                    err = True
                    break
        else: err = True
        if err:
            raise ValueError('x must be a 2d numpy array or a list of 2d numpy arrays')
        if type(y) != np.ndarray:
            raise ValueError('y must be a numpy array')
        if type(x) == np.ndarray:
            x = [x]
        # set variables

        super().__init__(batch_size=batch_size)
        self.x, self.y = x, y
        self.indices = np.arange(self.x[0].shape[0])
        self.n_inputs = len(x)
        self.shuffle = shuffle

    # required for instances of tf.keras.utils.Sequence
    def __len__(self):
        """Number of batches in the Sequence.
        """
        return math.ceil(self.x[0].shape[0] / self.batch_size)

    # required for instances of tf.keras.utils.Sequence
    def __getitem__(self, idx):
        """Gets batch at position `idx`.
        """
        # the starting and ending indices of the batch
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = []

        for i in range(self.n_inputs):
            batch_x.append(self.x[i][inds])
        batch_y = np.array(self.y[inds])
        return tuple(batch_x) , batch_y

    # required for instances of ktrain.Dataset
    def nsamples(self):
        return self.x[0].shape[0]

    #required for instances of ktrain.Dataset
    def get_y(self):
        return self.y

    def on_epoch_end(self):
        if self.shuffle:  np.random.shuffle(self.indices)

In [29]:
time_signature_train = np.expand_dims(cat_train_data.time_signature_cat, -1)
key_train = np.expand_dims(cat_train_data.key_cat, -1)
mode_train = np.expand_dims(cat_train_data.mode_cat, -1)

time_signature_valid = np.expand_dims(cat_valid_data.time_signature_cat, -1)
key_valid = np.expand_dims(cat_valid_data.key_cat, -1)
mode_valid = np.expand_dims(cat_valid_data.mode_cat, -1)

time_signature_test = np.expand_dims(cat_test_data.time_signature_cat, -1)
key_test = np.expand_dims(cat_test_data.key_cat, -1)
mode_test = np.expand_dims(cat_test_data.mode_cat, -1)

In [30]:
train_data = MyCustomDataset([num_train] + [trn2[0]] + [time_signature_train] + [key_train] + [mode_train], trn2[1])
valid_data = MyCustomDataset([num_valid] + [val2[0]] + [time_signature_valid] + [key_valid] + [mode_valid], val2[1])

# Building model

In [31]:
class NumericGenreClassifier:
    def __init__(self,num_train,cat_train,train_data,num_train_y,num_valid,cat_valid,valid_data,num_valid_y,class_weight,early_stop,experiment,text_model):
        # numeric and label input
        self.num_train = num_train
        self.num_train_y = num_train_y
        self.num_valid = num_valid
        self.num_valid_y = num_valid_y

        # categorical input
        self.cat_train = cat_train
        self.cat_valid = cat_valid

        # customed dataset
        self.train_data = train_data
        self.valid_data = valid_data

        # text model
        self.text_model = text_model

        # class weight and early stop mechanism
        self.class_weight = class_weight
        self.early_stop = early_stop

        # comet_ml experiment
        self.experiment = experiment
    
    def build_model(self):
        # numeric input layers
        numerical_input = Input(shape=(self.num_train.shape[1],), name='numerical_input')
        x = Dense(
            self.experiment.get_parameter("num_first_layer_units"),
            activation='elu',
            kernel_initializer='HeNormal')(numerical_input)
        x = Dropout(0.5)(x)
        x = BatchNormalization()(x)

        x = Dense(
            self.experiment.get_parameter("num_second_layer_units"),
            activation='elu',
            kernel_initializer='HeNormal')(x)
        x = Dropout(0.3)(x)
        x = BatchNormalization()(x)
        numerical_output = Flatten()(x) 

        # categorical input layers
        categorical_input = [time_signature_input,key_input,mode_input]
        concatenated = Concatenate(name='category')([time_signature_embedded, key_embedded, mode_embedded])
        embedded_input = Flatten()(concatenated)
        y = Dense(
            units=self.experiment.get_parameter("cat_first_layer_units"),
            activation='elu',
            kernel_initializer='HeUniform')(embedded_input)
        y = Dropout(0.5)(y)
        y = BatchNormalization()(y)

        y = Dense(
            units=self.experiment.get_parameter("cat_second_layer_units"),
            activation='elu',
            kernel_initializer='HeUniform')(y)
        y = Dropout(0.3)(y)
        y = BatchNormalization()(y)
        cat_out = y

        # final layer and output
        final_layer = Concatenate()([numerical_output, self.text_model.output, cat_out])
        final_layer = Dense(
                        units=self.experiment.get_parameter("final_layer_units"),
                        activation='elu',
                        kernel_initializer='HeNormal')(final_layer)
       
        final_model_output = Dense(8,activation='softmax', kernel_initializer='HeNormal')(final_layer)

        model = Model(inputs=[numerical_input] + [self.text_model.input] + [categorical_input],outputs=final_model_output,name='final_output')
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
    def train_model(self):
        # Train the model
        combined_model = self.build_model()
        model = ktrain.get_learner(
            combined_model,
            train_data=self.train_data,
            val_data=self.valid_data,
            batch_size=self.experiment.get_parameter("batch_size")
        )
        model.fit_onecycle(
            lr=0.001,
            epochs=self.experiment.get_parameter("epochs"),
            class_weight=self.class_weight,
        )
        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        # score = model.evaluate(self.num_valid,self.num_valid_y)
        # LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="music-genre-multiclass-classification"):

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

In [32]:
numerical_model = NumericGenreClassifier(num_train,cat_train,train_data,num_train_y,num_valid,cat_valid,valid_data,num_valid_y,class_weight,early_stop,experiment, model2)
# mu is the mean number of units, sigma is the standard deviation
numerical_model.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "num_first_layer_units": {"type": "discrete", "values": [700,750,800,850,900,950,1000]},
        "num_second_layer_units": {"type": "discrete", "values": [100,150,200,250,275,300,325,350]},
        "cat_first_layer_units": {"type": "discrete", "values": [200,250,300,325,350]},
        "cat_second_layer_units": {"type": "discrete", "values": [50,75,100,125,150,200]},
        "final_layer_units": {"type": "discrete", "values": [30,35,40,45,50,60,100,150]},
        "batch_size": {"type": "discrete", "values": [16,32,64]},
        "epochs": {"type": "discrete", "values": [6,7,8,9,10,11]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=f7eefc06439546e48265538a20abd146
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 241920, 'endTime': None, 'id': 'f7eefc06439546e48265538a20abd146', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [16, 32, 64]}, 'cat_first_layer_units': {'type': 'discrete', 'values': [200, 250, 300, 325, 350]}, 'cat_second_layer_units': {'type': 'discrete', 'values': [50, 75, 100, 125, 150, 200]}, 'epochs': {'type': 'discrete', 'values': [6, 7, 8, 9, 10, 11]}, 'final_layer_units': {'type': 'discrete', 'values': [30, 35, 40, 45, 50, 60, 100, 150]}, 'num_first_layer_units': {'type': 'discrete', 'values': [700, 750, 800, 850, 900, 950, 1000]}, 'num_second_layer_units': {'type': 'discrete', 'values': [100, 150, 200, 250, 275, 300, 325, 350]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleSize': 100, 'objec



begin training using onecycle policy with max lr of 0.001...
Epoch 1/6


COMET INFO: ignoring tensorflow summary log of metrics because of keras; set `comet_ml.loggers.tensorboard_logger.LOG_METRICS = True` to override


480/480 [==============================] - 7s 9ms/step - loss: 2.0233 - accuracy: 0.2954 - val_loss: 1.5048 - val_accuracy: 0.4444
Epoch 2/6
480/480 [==============================] - 4s 9ms/step - loss: 1.5768 - accuracy: 0.4073 - val_loss: 1.4253 - val_accuracy: 0.4956
Epoch 3/6
480/480 [==============================] - 4s 9ms/step - loss: 1.4351 - accuracy: 0.4716 - val_loss: 1.2579 - val_accuracy: 0.5689
Epoch 4/6
480/480 [==============================] - 5s 11ms/step - loss: 1.3159 - accuracy: 0.5082 - val_loss: 1.1479 - val_accuracy: 0.5978
Epoch 5/6
480/480 [==============================] - 6s 12ms/step - loss: 1.2206 - accuracy: 0.5395 - val_loss: 1.0852 - val_accuracy: 0.6244
Epoch 6/6
480/480 [==============================] - 5s 10ms/step - loss: 1.1425 - accuracy: 0.5607 - val_loss: 1.0280 - val_accuracy: 0.6511


begin training using onecycle policy with max lr of 0.001...
Epoch 1/6
480/480 [==============================] - 7s 12ms/step - loss: 2.0621 - accuracy: 0.313

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/2579eee6ef2c4437ae9746f9f9e9a8dd
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [12]                : (0.29538941383361816, 0.5814014077186584)
COMET INFO:     batch_accuracy [576]         : (0.0625, 0.75)
COMET INFO:     batch_loss [576]             : (1.0961949825286865, 3.2145023345947266)
COMET INFO:     epoch_duration [12]          : (4.25, 7.171999999991385)
COMET INFO:     loss [12]                    : (1.1023712158203125, 2.0620930194854736)
COMET INFO:     val_accuracy [12]            : (0.4444444477558136, 0.699999988079071)
COMET INFO:     val_loss [12]                : (0.9523516297340393, 1.5047643184661865)
COMET INFO:     validate_batch_accuracy [24] : (0.25,

...

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/2d4de599a7604db8a97a664aa762e24a
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [12]                : (0.32430320978164673, 0.628028154373169)
COMET INFO:     batch_accuracy [576]         : (0.10795454680919647, 0.6875)
COMET INFO:     batch_loss [576]             : (0.7355227470397949, 3.347062587738037)
COMET INFO:     epoch_duration [12]          : (4.7960000000020955, 6.342999999993481)
COMET INFO:     loss [12]                    : (0.973457396030426, 2.0121147632598877)
COMET INFO:     val_accuracy [12]            : (0.47555556893348694, 0.6844444274902344)
COMET INFO:     val_loss [12]                : (1.011400580406189, 1.4754425287246704)
COMET INFO:     validate_b



begin training using onecycle policy with max lr of 0.001...
Epoch 1/11
240/240 [==============================] - 4s 11ms/step - loss: 2.0957 - accuracy: 0.3097 - val_loss: 1.5437 - val_accuracy: 0.4111
Epoch 2/11
240/240 [==============================] - 2s 10ms/step - loss: 1.6158 - accuracy: 0.4230 - val_loss: 1.3535 - val_accuracy: 0.5000
Epoch 3/11
240/240 [==============================] - 2s 10ms/step - loss: 1.3854 - accuracy: 0.4852 - val_loss: 1.1788 - val_accuracy: 0.6133
Epoch 4/11
240/240 [==============================] - 2s 10ms/step - loss: 1.2589 - accuracy: 0.5322 - val_loss: 1.1733 - val_accuracy: 0.6200
Epoch 5/11
240/240 [==============================] - 2s 10ms/step - loss: 1.1332 - accuracy: 0.5754 - val_loss: 1.1370 - val_accuracy: 0.6267
Epoch 6/11
240/240 [==============================] - 2s 10ms/step - loss: 1.0664 - accuracy: 0.5944 - val_loss: 1.0710 - val_accuracy: 0.6378
Epoch 7/11
240/240 [==============================] - 2s 10ms/step - loss: 0.98

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/4d8f0b07b880438587b8044808b9d981
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [22]                : (0.29682210087776184, 0.6788226366043091)
COMET INFO:     batch_accuracy [528]         : (0.09375, 0.78125)
COMET INFO:     batch_loss [528]             : (0.49675679206848145, 2.9999091625213623)
COMET INFO:     epoch_duration [22]          : (2.327999999994063, 3.889999999999418)
COMET INFO:     loss [22]                    : (0.808866560459137, 2.0957329273223877)
COMET INFO:     val_accuracy [22]            : (0.41111111640930176, 0.6777777671813965)
COMET INFO:     val_loss [22]                : (1.0441083908081055, 1.543684482574463)
COMET INFO:     validate_batch_accu



begin training using onecycle policy with max lr of 0.001...
Epoch 1/6
120/120 [==============================] - 3s 15ms/step - loss: 2.1381 - accuracy: 0.2640 - val_loss: 1.5067 - val_accuracy: 0.4622
Epoch 2/6
120/120 [==============================] - 1s 12ms/step - loss: 1.5623 - accuracy: 0.4113 - val_loss: 1.3504 - val_accuracy: 0.5289
Epoch 3/6
120/120 [==============================] - 2s 17ms/step - loss: 1.3464 - accuracy: 0.4896 - val_loss: 1.2586 - val_accuracy: 0.5844
Epoch 4/6
120/120 [==============================] - 2s 17ms/step - loss: 1.1946 - accuracy: 0.5464 - val_loss: 1.2026 - val_accuracy: 0.6000
Epoch 5/6
120/120 [==============================] - 1s 11ms/step - loss: 1.0814 - accuracy: 0.5940 - val_loss: 1.1204 - val_accuracy: 0.6311
Epoch 6/6
120/120 [==============================] - 2s 15ms/step - loss: 1.0332 - accuracy: 0.6020 - val_loss: 1.1308 - val_accuracy: 0.6267


begin training using onecycle policy with max lr of 0.001...
Epoch 1/6
120/120 [===

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/045397ead8134190aa42d47e1ca0bf5d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [12]                : (0.2640010416507721, 0.6057567000389099)
COMET INFO:     batch_accuracy [144]         : (0.12215909361839294, 0.625)
COMET INFO:     batch_loss [144]             : (0.9200689196586609, 3.000833034515381)
COMET INFO:     epoch_duration [12]          : (1.0779999999940628, 2.9219999999913853)
COMET INFO:     loss [12]                    : (1.03323495388031, 2.151099443435669)
COMET INFO:     val_accuracy [12]            : (0.46222221851348877, 0.6399999856948853)
COMET INFO:     val_loss [12]                : (1.0916500091552734, 1.5194534063339233)
COMET INFO:     validate_ba

In [37]:
# numeric input layers
numerical_input = Input(shape=(num_train.shape[1],), name='numerical_input')
x = Dense(
    units=950,
    activation='elu',
    kernel_initializer='HeNormal')(numerical_input)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

x = Dense(
    units=275,
    activation='elu',
    kernel_initializer='HeNormal')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
numerical_output = Flatten()(x) 

# categorical input layers
categorical_input = [time_signature_input,key_input,mode_input]
concatenated = Concatenate(name='category')([time_signature_embedded, key_embedded, mode_embedded])
embedded_input = Flatten()(concatenated)
y = Dense(
    units=300,
    activation='elu',
    kernel_initializer='HeUniform')(embedded_input)
y = Dropout(0.5)(y)
y = BatchNormalization()(y)

y = Dense(
    units=75,
    activation='elu',
    kernel_initializer='HeUniform')(y)
y = Dropout(0.3)(y)
y = BatchNormalization()(y)
cat_out = y

# final layer and output
final_layer = Concatenate()([numerical_output, model2.output, cat_out])
final_layer = Dense(
                units=50,
                activation='elu',
                kernel_initializer='HeNormal')(final_layer)

final_model_output = Dense(8,activation='softmax', kernel_initializer='HeNormal')(final_layer)

model = Model(inputs=[numerical_input] + [model2.input] + [categorical_input],outputs=final_model_output,name='final_output')
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [38]:
learner = ktrain.get_learner(
    model,
    train_data=train_data,
    val_data=valid_data,
    batch_size=64
)
learner.fit_onecycle(
    lr=0.001,
    epochs=6,
    class_weight=class_weight,
)



begin training using onecycle policy with max lr of 0.001...
Epoch 1/6
120/120 [==============================] - 3s 14ms/step - loss: 2.0185 - accuracy: 0.3336 - val_loss: 1.5422 - val_accuracy: 0.4467
Epoch 2/6
120/120 [==============================] - 1s 10ms/step - loss: 1.5289 - accuracy: 0.4510 - val_loss: 1.3070 - val_accuracy: 0.5156
Epoch 3/6
120/120 [==============================] - 1s 10ms/step - loss: 1.2843 - accuracy: 0.5231 - val_loss: 1.2197 - val_accuracy: 0.5889
Epoch 4/6
120/120 [==============================] - 1s 10ms/step - loss: 1.1327 - accuracy: 0.5758 - val_loss: 1.2006 - val_accuracy: 0.6111
Epoch 5/6
120/120 [==============================] - 1s 10ms/step - loss: 1.0164 - accuracy: 0.6185 - val_loss: 1.1149 - val_accuracy: 0.6444
Epoch 6/6
120/120 [==============================] - 1s 10ms/step - loss: 0.9636 - accuracy: 0.6409 - val_loss: 1.1205 - val_accuracy: 0.6289


In [39]:
predictor = ktrain.get_predictor(model=learner.model, preproc=preproc2)

In [40]:
predictor.save('audio_meta_text_model')